# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need

from pyspark.sql.functions import col, count, rank, countDistinct, avg, when, lag, round, udf, sum, desc
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

In [2]:
# 2) instantiate a Spark session 
spark = SparkSession.builder \
    .appName("Sparkify") \
    .getOrCreate()
# 3) read in the data set located at the path "data/sparkify_log_small.json"
data_path = "data/sparkify_log_small.json"
df = spark.read.json(data_path)
# 4) write code to answer the quiz questions 

In [3]:
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Question 1

Which page did user id "" (empty string) NOT visit?

In [4]:
# TODO: write your code to answer question 1

df_blank_pages = df.filter(col("userId") == "").select('page').distinct()

df_q1 = df.select('page').distinct().subtract(df_blank_pages)
    
df_q1.show()

+----------------+
|            page|
+----------------+
|Submit Downgrade|
|       Downgrade|
|          Logout|
|   Save Settings|
|        Settings|
|        NextSong|
|         Upgrade|
|           Error|
|  Submit Upgrade|
+----------------+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [5]:
# TODO: use this space to explore the behavior of the user with an empty string

df.createOrReplaceTempView("df")

df_q2 = spark.sql("""
            SELECT DISTINCT page, auth
            FROM df
            WHERE userId = ''
            """)

df_q2.show()

+-----+----------+
| page|      auth|
+-----+----------+
|Login|Logged Out|
| Help|Logged Out|
| Home|     Guest|
| Home|Logged Out|
|About|Logged Out|
|About|     Guest|
+-----+----------+



# Question 3

How many female users do we have in the data set?

In [6]:
# TODO: write your code to answer question 3

df_q3 = df.where(col("gender") == "F").select(countDistinct("userId"))

df_q3.show()

+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                   462|
+----------------------+



# Question 4

How many songs were played from the most played artist?

In [7]:
# TODO: write your code to answer question 4

df_num_songs = df.groupBy("artist").agg(count("song").alias("num_songs"))

windowSpec = Window.orderBy(col("num_songs").desc())

df_q4 = df_num_songs.withColumn("rank", rank().over(windowSpec)).where(col("rank") == 1)

df_q4.show()

+--------+---------+----+
|  artist|num_songs|rank|
+--------+---------+----+
|Coldplay|       83|   1|
+--------+---------+----+



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [8]:
# TODO: write your code to answer question 5

function = udf(lambda ishome: int(ishome == 'Home'), IntegerType())

user_window = Window.partitionBy('userId').orderBy(desc('ts')).rangeBetween(Window.unboundedPreceding, 0)

cusum = df.filter((df.page == 'NextSong') | (df.page == 'Home')).select('userId', 'page', 'ts') \
          .withColumn('homevisit', function(col('page'))).withColumn('period', sum('homevisit').over(user_window))

result = cusum.filter((cusum.page == 'NextSong')).groupBy('userId', 'period').agg(count('period').alias('period_count'))

df_q5 = result.groupBy().agg(avg('period_count').alias('avg_period_count')) \
                       .withColumn('avg_period_count', round('avg_period_count').cast(IntegerType()))

df_q5.show()

+----------------+
|avg_period_count|
+----------------+
|               7|
+----------------+

